In [1]:
from datasets import load_dataset, DatasetDict

datasetPath = '../datasets/book-dataset/book-data.txt'
rawtextDataset = load_dataset("text", data_files=datasetPath)
rawtextDataset["train"][:5]

{'text': ['Teşekkürler',
  'Bu hikayeyi yazma sürecinde yanımda olan sınıf arkadaşlarıma, ',
  'Matematik Hocam Şükrü Tekin’e, ',
  'Bana fazlasıyla fikir veren Elif ve Nisa’ya,',
  'Her daim destek çıkan Semra, Ravza, Nisa ve Elif’e']}

In [2]:
def remove_empty_lines(batch):
    return {"text": [line for line in batch["text"] if line.strip()]}
def remove_empty_lines_sentence(batch):
    return {"text": [line.strip() for line in batch["text"]]}

filteredDataset = rawtextDataset.map(remove_empty_lines, batched=True)
filteredDataset = filteredDataset.map(remove_empty_lines_sentence, batched=True)
filteredDataset["train"][:5]

{'text': ['Teşekkürler',
  'Bu hikayeyi yazma sürecinde yanımda olan sınıf arkadaşlarıma,',
  'Matematik Hocam Şükrü Tekin’e,',
  'Bana fazlasıyla fikir veren Elif ve Nisa’ya,',
  'Her daim destek çıkan Semra, Ravza, Nisa ve Elif’e']}

In [3]:
splitDataset = filteredDataset["train"].train_test_split(test_size=0.1)

tokenizedDataset = DatasetDict({
    "train": splitDataset["train"],
    "validation": splitDataset["test"]
})
tokenizedDataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 291
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 33
    })
})

In [4]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenizeFunction(textFile):
    return tokenizer(textFile["text"], padding="max_length", truncation=True)

tokenizedDataset = tokenizedDataset.map(tokenizeFunction, batched=True)
dataCollator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
tokenizedDataset

Map:   0%|          | 0/291 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 291
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 33
    })
})

In [5]:
tokenizedDataset.set_format("torch")
tokenizedDataset.column_names
tokenizedDataset

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 291
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 33
    })
})

In [6]:
tokenizedDataset["validation"][:5]["input_ids"]

tensor([[  101,  1054,  1024,  ...,     0,     0,     0],
        [  101,  2521,  2175,  ...,     0,     0,     0],
        [  101,  1055,  1024,  ...,     0,     0,     0],
        [  101,  8915,  3366,  ...,     0,     0,     0],
        [  101,  9092, 11722,  ...,     0,     0,     0]])

In [10]:
tokenizedDataset["train"].column_names

['text', 'input_ids', 'token_type_ids', 'attention_mask']

In [11]:
print(tokenizedDataset["train"][0]["text"])

Burada tek kalmamak için üç tane kedi sahiplendim. Linda, Lina ve Mina. Linda British, Lina Smokin, Mina ise Ankara kedisiydi. O kadar uysallardı ki hemen birbirimize alışmıştık. Şimdi yeni evimde neredeyse tüm şehri gören balkonumda etrafı izlemeye devam edeyim. Erken uyumam gerekiyordu çünkü yarın ilk iş günümdü. Erkenden yatıp yarın işime gidecektim.


In [9]:
from torch.utils.data import DataLoader

trainDataLoader = DataLoader(tokenizedDataset["train"], shuffle=True, batch_size=8, collate_fn=dataCollator)
valDataLoader = DataLoader(tokenizedDataset["validation"], batch_size=8, collate_fn=dataCollator)

"""
for batch in trainDataLoader:
    break
{k: v.shape for k, v in batch.items()}"""

trainDataLoader